<a href="https://colab.research.google.com/github/Lizbethhespinosa/ModelosI/blob/main/04%20-%20modelo%20con%20preprocesado%20FeatureEng%2BLabelEncoding%20y%20Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Consideraciones antes de ejecutar el notebook


Se debe cambiar el entorno de ejecución para que el programa corra de manera correcta: clic en entorno de ejecucion -> cambiar tipo de entorno de ejecución -> se selecciona GPU T4 -> guardar.

* **Preprocesado FeatureEng:** porque realizamos Ingeniería de Características (Feature Engineering) al crear manualmente la variable SCORE_PRIVILEGIOS sumando los lujos. Esto no estaba en los datos originales.

* **LabelEncoding:** porque para poder meter los datos a los 3 modelos a la vez, convertimos todo el texto a números enteros (0, 1, 2...) usando LabelEncoder. (Si hubiéramos usado OneHot o Nativo, XGBoost o LightGBM habrían fallado o tardado más).

* **Ensemble:** porque la técnica de modelado no es un algoritmo individual, sino un Ensamble de Votación (Voting) que promedia las probabilidades de tres "cerebros" distintos, los cuales son Catboost, XGBoosty ligtGBM.

# Instalacion


In [1]:
# Instalamos las 3 librerías de potencia
!pip install catboost xgboost lightgbm -q

import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar Kaggle
data = {"username":"lizbethhespinosa","key":"07b7534e53b0fe2b8ec81ed287410d9e"}
with open('kaggle.json','w') as f: json.dump(data, f)
!chmod 600 kaggle.json
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

print("📥 Descargando datos...")
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -p /content/ --force
!unzip -o -q udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
print("✅ Entorno listo.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.0 MB/s eta 0:00:00
📥 Descargando datos...
  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.31GB/s]
✅ Entorno listo.


# Preprocesamiento Unificado (Label Encoding)

In [2]:
from sklearn.preprocessing import LabelEncoder

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
TARGET_COL = 'RENDIMIENTO_GLOBAL'
ID_COL = 'ID'

# Unir para Feature Engineering
df_all = pd.concat([df_train.drop(columns=[TARGET_COL]), df_test], axis=0)

# --- 1. FEATURE ENGINEERING (Las variables nuevas) ---
cols_tiene = [c for c in df_all.columns if 'TIENE' in c]
for col in cols_tiene:
    df_all[col] = df_all[col].astype(str).str.lower().map({'si': 1, 'yes': 1, 'no': 0}).fillna(0)
df_all['SCORE_PRIVILEGIOS'] = df_all[cols_tiene].sum(axis=1)

# --- 2. CODIFICACIÓN NUMÉRICA (LABEL ENCODING) ---
# XGBoost y LightGBM odian el texto. Convertimos todo a números.
categorical_cols = df_all.select_dtypes(include=['object']).columns.tolist()
if ID_COL in categorical_cols: categorical_cols.remove(ID_COL)

print("⚙️ Convirtiendo texto a números...")
for col in categorical_cols:
    df_all[col] = df_all[col].fillna("Missing").astype(str)
    le = LabelEncoder()
    df_all[col] = le.fit_transform(df_all[col])

# Rellenar numéricos restantes
numeric_cols = df_all.select_dtypes(include=['int64', 'float64']).columns.tolist()
for col in numeric_cols:
    df_all[col] = df_all[col].fillna(-999)

# Separar
X = df_all.iloc[:len(df_train)].drop(columns=[ID_COL], errors='ignore')
X_test_final = df_all.iloc[len(df_train):].drop(columns=[ID_COL], errors='ignore')
y = df_train[TARGET_COL]

# Codificar Target también
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

print(f"✅ Datos listos. Clases: {le_target.classes_}")

⚙️ Convirtiendo texto a números...
✅ Datos listos. Clases: ['alto' 'bajo' 'medio-alto' 'medio-bajo']


#Modelo 1 - CatBoost (GPU)

In [3]:
# Reinstalación rápida para evitar el error de CUDA driver

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

# Dividimos (Usamos stratify para mantener proporciones)
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.15, random_state=42, stratify=y_encoded)

print("🐯 Entrenando Modelo 1: CatBoost (Modo GPU)...")

model_cat = CatBoostClassifier(
    iterations=2500,
    learning_rate=0.04,
    depth=8,
    l2_leaf_reg=5,

    # --- CONFIGURACIÓN GPU ---
    task_type="GPU",
    devices='0',
    # -------------------------

    # Nota: No ponemos 'cat_features' aquí porque en el Ensamble
    # ya convertimos todo a números (LabelEncoding) en la celda 2.
    loss_function='MultiClass',
    eval_metric='Accuracy',
    random_seed=42,
    verbose=500,
    early_stopping_rounds=200
)

model_cat.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    use_best_model=True
)

# Guardamos PROBABILIDADES para el ensamble
probs_cat = model_cat.predict_proba(X_test_final)
print("✅ CatBoost terminado.")

🐯 Entrenando Modelo 1: CatBoost (Modo GPU)...
0:	learn: 0.3802166	test: 0.3801588	best: 0.3801588 (0)	total: 158ms	remaining: 6m 35s
500:	learn: 0.4377133	test: 0.4293526	best: 0.4293815 (490)	total: 16.5s	remaining: 1m 5s
1000:	learn: 0.4529471	test: 0.4323562	best: 0.4327413 (944)	total: 27.9s	remaining: 41.7s
1500:	learn: 0.4653829	test: 0.4332034	best: 0.4336462 (1420)	total: 39.2s	remaining: 26.1s
bestTest = 0.4337424789
bestIteration = 1536
Shrink model to first 1537 iterations.
✅ CatBoost terminado.


#Modelo 2 - XGBoost (GPU)

In [4]:
from xgboost import XGBClassifier

print("🚀 Entrenando Modelo 2: XGBoost (Modo GPU)...")

model_xgb = XGBClassifier(
    n_estimators=2500,    # Subimos iteraciones para igualar potencia
    learning_rate=0.04,
    max_depth=8,
    subsample=0.7,
    colsample_bytree=0.7,

    # --- CONFIGURACIÓN GPU ---
    tree_method='hist',   # Método histograma (el más rápido)
    device='cuda',        # Usar la GPU de Colab
    # -------------------------

    random_state=42,
    early_stopping_rounds=100
)

# Entrenar
model_xgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=500
)

probs_xgb = model_xgb.predict_proba(X_test_final)
print("✅ XGBoost terminado.")

🚀 Entrenando Modelo 2: XGBoost (Modo GPU)...
[0]	validation_0-mlogloss:1.37980
[500]	validation_0-mlogloss:1.20014
[902]	validation_0-mlogloss:1.19912


/usr/local/lib/python3.12/dist-packages/xgboost/core.py:774: UserWarning: [13:35:00] WARNING: /workspace/src/common/error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


✅ XGBoost terminado.


#Modelo 3 - LightGBM (CPU Optimizado)

In [5]:
from lightgbm import LGBMClassifier

print("💡 Entrenando Modelo 3: LightGBM...")
model_lgb = LGBMClassifier(
    n_estimators=2000,
    learning_rate=0.05,
    num_leaves=31, # Típico de LightGBM
    random_state=42,
    n_jobs=-1
)

model_lgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='multi_logloss',
    callbacks=[
        # LightGBM usa callbacks para verbose en versiones nuevas,
        # si falla, simplemente quita el argumento callbacks
    ]
)

probs_lgb = model_lgb.predict_proba(X_test_final)
print("✅ LightGBM terminado.")

💡 Entrenando Modelo 3: LightGBM...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.144820 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1238
[LightGBM] [Info] Number of data points in the train set: 588625, number of used features: 20
[LightGBM] [Info] Start training from score -1.371992
[LightGBM] [Info] Start training from score -1.387091
[LightGBM] [Info] Start training from score -1.395032
[LightGBM] [Info] Start training from score -1.391214
✅ LightGBM terminado.


#EL ENSAMBLE (Soft Voting)

In [6]:
print("⚖️ Calculando el Ensamble (Promedio Ponderado)...")

# Podemos dar pesos. CatBoost y XGBoost suelen ser mejores, les damos más peso.
# Pesos: CatBoost (40%) + XGBoost (40%) + LightGBM (20%)
final_probs = (probs_cat * 0.4) + (probs_xgb * 0.4) + (probs_lgb * 0.2)

# Elegimos la clase con mayor probabilidad promedio
final_preds_indices = np.argmax(final_probs, axis=1)
final_preds_texto = le_target.inverse_transform(final_preds_indices)

# Crear archivo
if ID_COL in df_test.columns:
    test_ids = df_test[ID_COL]
else:
    test_ids = df_test.index

submission = pd.DataFrame({
    ID_COL: test_ids,
    TARGET_COL: final_preds_texto
})

out_filename = "submission_07_Ensemble_Voting.csv"
submission.to_csv(out_filename, index=False)
print(f"💾 Archivo Final Generado: {out_filename}")



⚖️ Calculando el Ensamble (Promedio Ponderado)...
💾 Archivo Final Generado: submission_07_Ensemble_Voting.csv


#Subir


In [7]:
message = "Notebook 07 - Ensemble (CatBoost + XGBoost + LightGBM)"
!kaggle competitions submit -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -f {out_filename} -m "{message}"

print("🏆 ¡Subida lista! Este debería ser tu mejor puntaje.")

100% 4.04M/4.04M [00:03<00:00, 1.29MB/s]
Successfully submitted to UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia🏆 ¡Subida lista! Este debería ser tu mejor puntaje.
